<a href="https://colab.research.google.com/github/Inferno-P/WISE_Assignment/blob/master/Wise_Assignment_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=6109ea1e154db2dce58bec197578a6f803e6bb2f5a4d483c567b9b9ca819a86d
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


# Theory

I send 100 GBP to Euro when 0.84 EUR = 1 GBP. 


| Rate 	| Amount Send (in GBP) 	| Amount Received (in EUR) 	| Gain and Loss 	| Remark                      	|
|------	|----------------------	|--------------------------	|---------------	|-----------------------------	|
| 0.84 	| 100                  	| 84                       	| 0             	| Original Transaction        	|
| 0.85 	| 100                  	| 85                       	| +1            	| Bob gets paid one more euro 	|
| 0.82 	| 100                  	| 82                       	| -2            	| Bob gets paid two less euro 	|
| 0.81 	| 100                  	| 81                       	| -3            	|                             	|
| 0.79 	| 100                  	| 79                       	| -5            	|                             	|

So, when the transaction gets settled means a lot for Bob as he can gain or lose. 



> The Exposure is a chance of Loss or Gain.

> Transaction exposure is the **risk of loss from a change in exchange rates during the course of a business transaction**. This exposure is derived from changes in foreign exchange rates between the dates when a transaction is booked and when it is settled.



In [ ]:
!ls -a
#https://docs.google.com/spreadsheets/d/1K3wlD2niOsfW8QSJmsIsClxdrVPC32gW/edit?usp=sharing&ouid=110112903238340246673&rtpof=true&sd=true

.  ..  .config	sample_data  Treasury_Analyst_Assignment.xlsx


In [ ]:
import pandas as pd

vol_data = pd.read_excel("Treasury_Analyst_Assignment.xlsx",sheet_name='Volume')
rates_data = pd.read_excel("Treasury_Analyst_Assignment.xlsx",sheet_name='Rates')
hedge_data = pd.read_excel("Treasury_Analyst_Assignment.xlsx",sheet_name='Hedge')
display(vol_data.head(), rates_data.head(), hedge_data.head())

,TransferId,TimeStamp,FromCcy,ToCcy,FromCcyAmount
0,3456765.0,2017-09-05 12:00:00,EUR,GBP,5900.0
1,3456766.0,2017-09-05 12:03:00,GBP,EUR,5500.0
2,3456767.0,2017-09-05 12:03:53,EUR,GBP,6500.0
3,3456768.0,2017-09-05 12:09:22,EUR,GBP,5400.0
4,3456769.0,2017-09-05 12:10:55,GBP,EUR,5200.0


,TimeStamp,EUR/GBP
0,2017-09-05 12:00:00,0.8474
1,2017-09-05 12:03:00,0.8481
2,2017-09-05 12:03:53,0.8472
3,2017-09-05 12:09:22,0.8477
4,2017-09-05 12:10:55,0.8485


,TimeStamp,Ccy,Amount
0,2017-09-05 15:06:00,EUR,40000.0
1,2017-09-05 16:45:00,EUR,-20000.0
2,2017-09-05 17:20:00,EUR,10000.0
3,2017-09-05 20:27:00,EUR,10000.0
4,2017-09-05 21:37:00,EUR,20000.0


In [ ]:
rates_data.shape, vol_data.shape
#Standardising rates
vol_data['TimeStamp'] = pd.to_datetime(vol_data['TimeStamp'])
rates_data['TimeStamp'] = pd.to_datetime(rates_data['TimeStamp'])
hedge_data['TimeStamp'] = pd.to_datetime(hedge_data['TimeStamp'])

In [ ]:
vol_rate_data = pd.merge(vol_data,
                         rates_data,
                         how='left',
                         on='TimeStamp')
vol_rate_data.head()

,TransferId,TimeStamp,FromCcy,ToCcy,FromCcyAmount,EUR/GBP
0,3456765.0,2017-09-05 12:00:00,EUR,GBP,5900.0,0.8474
1,3456766.0,2017-09-05 12:03:00,GBP,EUR,5500.0,0.8481
2,3456767.0,2017-09-05 12:03:53,EUR,GBP,6500.0,0.8472
3,3456768.0,2017-09-05 12:09:22,EUR,GBP,5400.0,0.8477
4,3456769.0,2017-09-05 12:10:55,GBP,EUR,5200.0,0.8485


In [ ]:
vol_rate_data['AmountGBP'] = vol_rate_data['FromCcyAmount'] * vol_rate_data['EUR/GBP']
vol_rate_data.head()

,TransferId,TimeStamp,FromCcy,ToCcy,FromCcyAmount,EUR/GBP,AmountGBP
0,3456765.0,2017-09-05 12:00:00,EUR,GBP,5900.0,0.8474,4999.66
1,3456766.0,2017-09-05 12:03:00,GBP,EUR,5500.0,0.8481,4664.55
2,3456767.0,2017-09-05 12:03:53,EUR,GBP,6500.0,0.8472,5506.80
3,3456768.0,2017-09-05 12:09:22,EUR,GBP,5400.0,0.8477,4577.58
4,3456769.0,2017-09-05 12:10:55,GBP,EUR,5200.0,0.8485,4412.20


In [ ]:
vol_rate_data['EUR_Position'] = vol_rate_data.apply(lambda row: row.FromCcyAmount if row.FromCcy == "EUR" else (-1)* row.FromCcyAmount , axis=1)

vol_rate_data.head()

,TransferId,TimeStamp,FromCcy,ToCcy,FromCcyAmount,EUR/GBP,AmountGBP,EUR_Position
0,3456765.0,2017-09-05 12:00:00,EUR,GBP,5900.0,0.8474,4999.66,5900.0
1,3456766.0,2017-09-05 12:03:00,GBP,EUR,5500.0,0.8481,4664.55,-5500.0
2,3456767.0,2017-09-05 12:03:53,EUR,GBP,6500.0,0.8472,5506.80,6500.0
3,3456768.0,2017-09-05 12:09:22,EUR,GBP,5400.0,0.8477,4577.58,5400.0
4,3456769.0,2017-09-05 12:10:55,GBP,EUR,5200.0,0.8485,4412.20,-5200.0


In [ ]:
vol_rate_data['Total_EUR_Exposure'] = 
vol_rate_data['Net_EUR_Exposure'] = vol_rate_data['EUR_Position'].cumsum()
vol_rate_data.head()

,TransferId,TimeStamp,FromCcy,ToCcy,FromCcyAmount,EUR/GBP,AmountGBP,EUR_Position,Net_EUR_Exposure
0,3456765.0,2017-09-05 12:00:00,EUR,GBP,5900.0,0.8474,4999.66,5900.0,5900.0
1,3456766.0,2017-09-05 12:03:00,GBP,EUR,5500.0,0.8481,4664.55,-5500.0,400.0
2,3456767.0,2017-09-05 12:03:53,EUR,GBP,6500.0,0.8472,5506.80,6500.0,6900.0
3,3456768.0,2017-09-05 12:09:22,EUR,GBP,5400.0,0.8477,4577.58,5400.0,12300.0
4,3456769.0,2017-09-05 12:10:55,GBP,EUR,5200.0,0.8485,4412.20,-5200.0,7100.0


In [ ]:
vol_rate_data.head()

,TransferId,TimeStamp,FromCcy,ToCcy,FromCcyAmount,EUR/GBP,AmountGBP,EUR_Exposure,EUR_Position,Net_EUR_Exposure
0,3456765.0,2017-09-05 12:00:00,EUR,GBP,5900.0,0.8474,4999.66,5900.0,5900.0,5900.0
1,3456766.0,2017-09-05 12:03:00,GBP,EUR,5500.0,0.8481,4664.55,400.0,-5500.0,400.0
2,3456767.0,2017-09-05 12:03:53,EUR,GBP,6500.0,0.8472,5506.80,6900.0,6500.0,6900.0
3,3456768.0,2017-09-05 12:09:22,EUR,GBP,5400.0,0.8477,4577.58,12300.0,5400.0,12300.0
4,3456769.0,2017-09-05 12:10:55,GBP,EUR,5200.0,0.8485,4412.20,7100.0,-5200.0,7100.0
